In [1]:
import keras
model_original = keras.models.load_model("mnist_cnn_allconv.h5")

Using TensorFlow backend.


In [9]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (4, 4)

input_shape = (28, 28, 1)

model = Sequential()

model.add(Convolution2D(32, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape,
                        subsample=pool_size))
model.add(Activation('relu'))
model.add(Convolution2D(64, kernel_size[0], kernel_size[1],
                        border_mode='valid', subsample=pool_size))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 13, 13, 32)    544         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 13, 13, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 5, 5, 64)      32832       activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 5, 5, 64)      0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [35]:
from keras import backend as K
from keras.utils.np_utils import convert_kernel
import tensorflow as tf
import numpy as np
ops = []
#convert the conv layers
for old_layer, new_layer in zip(model_original.layers, model.layers):
    if old_layer.__class__.__name__ in ['Convolution2D', 'Convolution1D']:
        original_w, original_b = old_layer.get_weights()
        #transpose it; it's currently in outchan,inchan,rows,cols
        #make it rows,cols,inchan,outchan
        transposed_w = np.transpose(original_w[:,:,::-1,::-1], (2,3,1,0))
        new_layer.set_weights([transposed_w, original_b])

In [91]:
#convert the dense layer
dense_w, dense_b = model_original.layers[6].get_weights()
print(dense_w.shape)
reshaped_dense_w = np.reshape(dense_w, (64,5,5,128))
axes_swapped_dense_w = np.transpose(reshaped_dense_w,(1,2,0,3))
new_dense_w = np.reshape(axes_swapped_dense_w, (1600,128))
model.layers[6].set_weights([new_dense_w, dense_b])

#second-to-last dense layer just has regular weights
model.layers[9].set_weights(model_original.layers[9].get_weights())

(1600, 128)


In [92]:
model.save('mnist_cnn_allconv_tensorflow.h5')

In [93]:
np.random.seed(1234)
vals = np.random.random((1,28,28,1))
print(model.predict([vals]))
from keras.models import Model
layer_model = Model(input=model.input, output=model.layers[9].output)
print(np.sum(layer_model.predict([vals])))
print(layer_model.predict([vals]))

[[ 0.04705426  0.03339526  0.0796259   0.17400834  0.01364966  0.04841387
   0.02714486  0.01118005  0.53199524  0.03353257]]
-5.71005
[[-0.60911679 -0.95200425 -0.08307897  0.69868487 -1.84670365 -0.58063215
  -1.15923071 -2.04628801  1.81621611 -0.94790119]]
